In [1]:
import pandas as pd

import numpy as np

h5_file = 'G:\\PROJECTS\\CAEP1073 - EIS PC-12 Radar Installation\\wip\\FEM\\DFEM PostMod - M3\\Run_M3.0\\sol101_ult_Incl_Radome\\pc12-47-v1-6-dfem_post_m3.0_ult_incl_radome.h5'

store = pd.HDFStore(h5_file)

In [4]:
def get_df(x, eid_array, cols = list('AB')):
    
    df = pd.DataFrame(x[:,[0,10]],index=eid_array, columns=cols)
    
    return df

# In[12]:


def Join_DF(df1,df2):
    
    if df1.empty == True:
        
        df1 = df2.copy()
        
    else :
        
        df1 = pd.concat([df1, df2], axis=1, sort=False, join_axes=[df1.index])
        
#         df1 = df1.join(df2, sort=False)
        
        
    return df1

# In[13]:


def CBAR_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BAR')
    
    return df

# In[14]:


def CBEAM_ELEMENTAL_FORCES(store):
    
    beam_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BEAM')

    beam_lst_names = beam_node.colnames[1:-1]

    res_df = pd.DataFrame({'A' : []})

    for x in beam_lst_names:

        col_name_a = x + '_A'

        col_name_b = x + '_B'

        df = get_df(beam_node.col(x), beam_node.col('EID'), cols=[col_name_a, col_name_b])

        res_df = Join_DF(res_df, df)
        
    domain_id_df = pd.DataFrame(beam_node.col('DOMAIN_ID'), index=beam_node.col('EID'), columns=['DOMAIN_ID'])
    
    res_df = Join_DF(res_df, domain_id_df)
    
    res_df.index.name = 'EID'
    
    return res_df

# In[15]:


def CELAS2_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ELAS2')
    
    return df


# In[16]:


def CQUAD4_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/QUAD4')
    
    return df


# In[17]:


def CROD_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/ROD')
    
    return df


# In[18]:


def CBUSH_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/BUSH')
    
    return df


def CSHEAR_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/SHEAR')
    
    return df


def CTRIA3_ELEMENTAL_FORCES(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/ELEMENT_FORCE/TRIA3')
    
    return df


def CBAR_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/BAR')
    
    return df


def CBEAM_ELEMENTAL_STRESS(store):
    
    beam_node = store.get_node('/NASTRAN/RESULT/ELEMENTAL/STRESS/BEAM')

    beam_lst_names = beam_node.colnames[1:-1]

    res_df = pd.DataFrame({'A' : []})

    for x in beam_lst_names:

        col_name_a = x + '_A'

        col_name_b = x + '_B'

        df = get_df(beam_node.col(x), beam_node.col('EID'), cols=[col_name_a, col_name_b])

        res_df = Join_DF(res_df, df)
        
    domain_id_df = pd.DataFrame(beam_node.col('DOMAIN_ID'), index=beam_node.col('EID'), columns=['DOMAIN_ID'])
    
    res_df.index.name = 'EID'
    
    res_df = Join_DF(res_df, domain_id_df)
    
    return res_df


def CELAS2_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/ELAS2')
    
    return df

def CQUAD4_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/QUAD4')
    
    return df


def CROD_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/ROD')
    
    return df

def CSHEAR_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/SHEAR')
    
    return df

def CTRIA3_ELEMENTAL_STRESS(store):
    
    df = store.get('/NASTRAN/RESULT/ELEMENTAL/STRESS/TRIA3')
    
    return df

In [6]:
def get_subcase_mapping(store):
        
    domain_results  =  store.get('/NASTRAN/RESULT/DOMAINS')

    domain_results = domain_results.loc[:, ['ID', 'SUBCASE']]

    domain_results.set_index('ID', inplace=True)

    subcase_mapping = domain_results.to_dict()

    return subcase_mapping

In [7]:
cbush_df = CBUSH_ELEMENTAL_FORCES(store)

In [8]:
subcase_mapping = get_subcase_mapping(store)

cbush_df['SUBCASE'] = cbush_df.DOMAIN_ID.map(subcase_mapping['SUBCASE'])

In [9]:
def filter_cbush_results(df, filter_df):
    
    df['isIn'] = df['EID'].isin(filter_df['elements'].tolist()).copy()
    
    df1 = df[df['isIn'] == True].copy()
    
    df1.drop(['isIn', 'DOMAIN_ID'], axis=1, inplace = True)
    
    return df1

In [ ]:
cbush_df.index.name = 'Index'

cbush_df = cbush_df[['EID', 'SUBCASE', 'FX', 'FY', 'FZ', 'MX', 'MY', 'MZ']]

cbush_df.to_csv('cbush_loads_extractions.csv')

In [16]:
store.close()